# Hybrid Probe Validation Notebook

This notebook provides an interactive environment to validate the hybrid probe creation tool.
It demonstrates:
1. Loading probes from different sources
2. Creating a hybrid probe by mixing amplitude and phase
3. Visual comparison of source and hybrid probes
4. Numerical validation of the mixing process

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import os

# Add project root to path
project_root = Path('.').absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import our utilities
from ptycho.workflows.simulation_utils import load_probe_from_source
from scripts.tools.create_hybrid_probe import create_hybrid_probe

# Set up plotting
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

## Helper Functions

In [ ]:
def display_probe(probe, title="Probe", figsize=(10, 4)):
    """Display a probe with amplitude and phase subplots."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Amplitude
    im1 = ax1.imshow(np.abs(probe), cmap='gray')
    ax1.set_title(f"{title} - Amplitude")
    ax1.axis('off')
    plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
    
    # Phase
    im2 = ax2.imshow(np.angle(probe), cmap='hsv', vmin=-np.pi, vmax=np.pi)
    ax2.set_title(f"{title} - Phase")
    ax2.axis('off')
    cbar = plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
    cbar.set_label('Phase (radians)')
    
    plt.tight_layout()
    return fig

def probe_statistics(probe, name="Probe"):
    """Print statistics about a probe."""
    print(f"\n{name} Statistics:")
    print(f"  Shape: {probe.shape}")
    print(f"  Dtype: {probe.dtype}")
    print(f"  Amplitude range: [{np.min(np.abs(probe)):.4f}, {np.max(np.abs(probe)):.4f}]")
    print(f"  Phase range: [{np.min(np.angle(probe)):.4f}, {np.max(np.angle(probe)):.4f}]")
    print(f"  Total power: {np.sum(np.abs(probe)**2):.4f}")
    print(f"  Contains NaN: {np.any(np.isnan(probe))}")
    print(f"  Contains Inf: {np.any(np.isinf(probe))}")

## 1. Create Test Probes

For demonstration, we'll create synthetic test probes. In practice, you would load these from actual datasets.

In [ ]:
# Create synthetic test probes
size = 128
x = np.linspace(-2, 2, size)
y = np.linspace(-2, 2, size)
X, Y = np.meshgrid(x, y)
R = np.sqrt(X**2 + Y**2)

# Probe 1: Gaussian amplitude with flat phase
probe1 = np.exp(-R**2).astype(np.complex64)

# Probe 2: Flat amplitude with quadratic phase (aberrated)
phase_aberration = 2 * R**2 + 0.5 * X  # Quadratic + tilt
probe2 = np.exp(1j * phase_aberration).astype(np.complex64)

# Display both probes
display_probe(probe1, "Probe 1 (Amplitude Source)")
probe_statistics(probe1, "Probe 1")

display_probe(probe2, "Probe 2 (Phase Source)")
probe_statistics(probe2, "Probe 2")

## 2. Create Hybrid Probe

Now we'll create a hybrid probe that combines the amplitude from Probe 1 with the phase from Probe 2.

In [ ]:
# Create hybrid probe
hybrid = create_hybrid_probe(probe1, probe2, normalize=False)

# Display result
display_probe(hybrid, "Hybrid Probe")
probe_statistics(hybrid, "Hybrid")

## 3. Side-by-Side Comparison

Let's create a comprehensive comparison showing all three probes together.

In [ ]:
# Create comparison figure
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Probe 1
im = axes[0, 0].imshow(np.abs(probe1), cmap='gray')
axes[0, 0].set_title('Probe 1\n(Amplitude)', fontsize=12)
plt.colorbar(im, ax=axes[0, 0], fraction=0.046)

im = axes[1, 0].imshow(np.angle(probe1), cmap='hsv', vmin=-np.pi, vmax=np.pi)
axes[1, 0].set_title('Probe 1\n(Phase)', fontsize=12)
plt.colorbar(im, ax=axes[1, 0], fraction=0.046)

# Probe 2
im = axes[0, 1].imshow(np.abs(probe2), cmap='gray')
axes[0, 1].set_title('Probe 2\n(Amplitude)', fontsize=12)
plt.colorbar(im, ax=axes[0, 1], fraction=0.046)

im = axes[1, 1].imshow(np.angle(probe2), cmap='hsv', vmin=-np.pi, vmax=np.pi)
axes[1, 1].set_title('Probe 2\n(Phase)', fontsize=12)
plt.colorbar(im, ax=axes[1, 1], fraction=0.046)

# Hybrid
im = axes[0, 2].imshow(np.abs(hybrid), cmap='gray')
axes[0, 2].set_title('Hybrid Probe\n(Amp from 1)', fontsize=12)
plt.colorbar(im, ax=axes[0, 2], fraction=0.046)

im = axes[1, 2].imshow(np.angle(hybrid), cmap='hsv', vmin=-np.pi, vmax=np.pi)
axes[1, 2].set_title('Hybrid Probe\n(Phase from 2)', fontsize=12)
plt.colorbar(im, ax=axes[1, 2], fraction=0.046)

for ax in axes.flat:
    ax.axis('off')

plt.suptitle('Hybrid Probe Creation Validation', fontsize=16)
plt.tight_layout()

## 4. Numerical Validation

Let's verify that the hybrid probe correctly combines amplitude from source 1 and phase from source 2.

In [ ]:
# Validation checks
validation_results = []

# Check 1: Amplitude matches source 1
amp_diff = np.abs(hybrid) - np.abs(probe1)
amp_error = np.max(np.abs(amp_diff))
amp_match = amp_error < 1e-6
validation_results.append(('Amplitude matches source 1', amp_match, f"Max error: {amp_error:.2e}"))

# Check 2: Phase matches source 2
phase_diff = np.angle(np.exp(1j * (np.angle(hybrid) - np.angle(probe2))))
phase_error = np.max(np.abs(phase_diff))
phase_match = phase_error < 1e-6
validation_results.append(('Phase matches source 2', phase_match, f"Max error: {phase_error:.2e}"))

# Check 3: No NaN/Inf values
no_nan = not np.any(np.isnan(hybrid))
no_inf = not np.any(np.isinf(hybrid))
finite_values = no_nan and no_inf
validation_results.append(('All values finite', finite_values, f"NaN: {not no_nan}, Inf: {not no_inf}"))

# Check 4: Correct dtype
correct_dtype = hybrid.dtype == np.complex64
validation_results.append(('Dtype is complex64', correct_dtype, f"Actual: {hybrid.dtype}"))

# Display results
print("\nValidation Results:")
print("-" * 60)
for test_name, passed, details in validation_results:
    status = "✓ PASS" if passed else "✗ FAIL"
    print(f"{status} | {test_name:<30} | {details}")

# Overall result
all_passed = all(result[1] for result in validation_results)
print("-" * 60)
print(f"\nOverall: {'✓ ALL TESTS PASSED' if all_passed else '✗ SOME TESTS FAILED'}")

## 5. Test with Different Probe Sizes

The tool should handle probes of different sizes by resizing to match the larger one.

In [ ]:
# Create probes with different sizes
size1 = 64
size2 = 128

# Small probe (amplitude source)
x1 = np.linspace(-2, 2, size1)
X1, Y1 = np.meshgrid(x1, x1)
R1 = np.sqrt(X1**2 + Y1**2)
small_probe = np.exp(-R1**2).astype(np.complex64)

# Large probe (phase source)
x2 = np.linspace(-2, 2, size2)
X2, Y2 = np.meshgrid(x2, x2)
R2 = np.sqrt(X2**2 + Y2**2)
large_probe = np.exp(1j * R2**2).astype(np.complex64)

print(f"Small probe shape: {small_probe.shape}")
print(f"Large probe shape: {large_probe.shape}")

# Create hybrid
hybrid_resized = create_hybrid_probe(small_probe, large_probe)
print(f"\nHybrid probe shape: {hybrid_resized.shape}")
print(f"Resizing successful: {hybrid_resized.shape == large_probe.shape}")

# Display
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(np.abs(small_probe), cmap='gray')
axes[0].set_title(f'Small Amplitude\n{small_probe.shape}')
axes[0].axis('off')

axes[1].imshow(np.angle(large_probe), cmap='hsv', vmin=-np.pi, vmax=np.pi)
axes[1].set_title(f'Large Phase\n{large_probe.shape}')
axes[1].axis('off')

axes[2].imshow(np.abs(hybrid_resized), cmap='gray')
axes[2].set_title(f'Hybrid (Resized)\n{hybrid_resized.shape}')
axes[2].axis('off')

plt.tight_layout()

## 6. Test Power Normalization

The tool can optionally normalize the hybrid probe to preserve the total power from the amplitude source.

In [ ]:
# Create probe with high power
high_power_probe = 3.0 * probe1  # 3x amplitude

# Create hybrid with and without normalization
hybrid_no_norm = create_hybrid_probe(high_power_probe, probe2, normalize=False)
hybrid_norm = create_hybrid_probe(high_power_probe, probe2, normalize=True)

# Calculate powers
power_original = np.sum(np.abs(high_power_probe)**2)
power_no_norm = np.sum(np.abs(hybrid_no_norm)**2)
power_norm = np.sum(np.abs(hybrid_norm)**2)

print("Power Comparison:")
print(f"Original probe power: {power_original:.4f}")
print(f"Hybrid (no normalization) power: {power_no_norm:.4f}")
print(f"Hybrid (normalized) power: {power_norm:.4f}")
print(f"\nPower preserved with normalization: {np.abs(power_norm - power_original) < 1e-3}")

# Visual comparison
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

im1 = axes[0].imshow(np.abs(high_power_probe), cmap='gray')
axes[0].set_title(f'Original\nPower: {power_original:.1f}')
plt.colorbar(im1, ax=axes[0], fraction=0.046)

im2 = axes[1].imshow(np.abs(hybrid_no_norm), cmap='gray')
axes[1].set_title(f'No Normalization\nPower: {power_no_norm:.1f}')
plt.colorbar(im2, ax=axes[1], fraction=0.046)

im3 = axes[2].imshow(np.abs(hybrid_norm), cmap='gray')
axes[2].set_title(f'Normalized\nPower: {power_norm:.1f}')
plt.colorbar(im3, ax=axes[2], fraction=0.046)

for ax in axes:
    ax.axis('off')

plt.tight_layout()

## 7. Command-Line Example

Here's how you would use the tool from the command line:

In [ ]:
print("Command-line usage examples:\n")

print("1. Basic usage:")
print("   python scripts/tools/create_hybrid_probe.py amplitude_probe.npy phase_probe.npy\n")

print("2. With custom output and visualization:")
print("   python scripts/tools/create_hybrid_probe.py \\")
print("       datasets/default_probe.npz \\")
print("       datasets/fly64/fly001_64_train_converted.npz \\")
print("       --output hybrid_probe_fly64.npy \\")
print("       --visualize\n")

print("3. With power normalization:")
print("   python scripts/tools/create_hybrid_probe.py \\")
print("       amp_source.npy phase_source.npy \\")
print("       --output normalized_hybrid.npy \\")
print("       --normalize \\")
print("       --visualize")

## Summary

This notebook has demonstrated:

1. ✓ The hybrid probe tool correctly combines amplitude from one source with phase from another
2. ✓ The tool handles probes of different sizes by resizing to the larger dimension
3. ✓ Power normalization works correctly when enabled
4. ✓ All outputs are finite and have the correct complex64 dtype
5. ✓ The tool can be used both programmatically and from the command line

The hybrid probe creation tool is ready for use in controlled ptychography experiments!